# 2023年度 深層学習演習 最終課題
## 23vr008n 高林 秀

## 問題設定
### 選択した課題
* (a) : テーブルデータ、画像データ、系列データのいずれか(※ 広い意味で捉えて良い) を用意し、回帰もしくは分類タスクを設定して深層学習モデルにより学習、評価を行え。よりよい結果が得られるよう試行錯誤し、過程も含めて示すこと。 
### 設定したタスク
* ポケモンの画像分類-オーキドモデルを作る-
 * 下記データセットを使用する。
 * 149クラスある
* https://www.kaggle.com/datasets/echometerhhwl/pokemon-gen-1-38914

## データセットについて
学習で使うデータセットはkaggleからダウンロードしたものを使用する。

このデータは、**149種類のポケモンの画像**が入っている。アニメ画像の切り抜きや、手書きで描いたポケモンの画像などが含まれている。

各ポケモン毎にフォルダが分かれており、その中に画像が入っている。
画像の層数は全部で**35627枚**である。

## 使用するモデルについて
本課題では事前学習済みのモデルをファインチューニングすることで、モデルを作成する。

今回使用するモデルは以下の通りである。
* モデル名：EfficientNetV2
* TODO: モデルの説明を書く

## 評価方法
ダウンロードしたデータセットを、学習用とテストセットに分け、テストセットを用いて評価を行う。

なお、評価方法は、**Accuracy**を用いる。

## 実行環境
本稿で作成したモデルは、ローカルマシン上で構築した。以下に実行環境を示す。

* OS: Windows 11
* CPU: Intel(R) Core(TM) i7-13700H @ 2.40GHz
* GPU: NVIDIA GeForce RTX 4060 Laptop GPU @ 8GB
* RAM: 32GB
* 開発環境
    * Python 3.9.6
    * PyTorch 1.9.1 + cu111
    * Torchvision 0.10.1 + cu111

## サンプルデータの表示
学習データで使用する画像は次のようなもので、単純にポケモンのみが写っているものもあれば、背景に人間が写っているものや、手書きのもの、カード状のものまで様々である。

<img src="archive/data/Raichu/0c7e0a91bf65facbbc2d2c06b55f1bf2.jpg" style="width: 250px;"/>
<img src="archive/data/Raichu/ee730b7e1e47a1aafd94476016875344.jpg" style="width: 250px;"/>
<img src="archive/data/Vileplume/0ac8fd551dd66cda9f03e6124363b071.png" style="width: 250px;"/>
<img src="archive/data/Zapdos/0b3cf9fd603a437e50e76222ce0db245.jpg" style="width: 250px;"/>


In [4]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [18]:
#CUDAが有効化どうかの確認
torch.cuda.is_available()

True

In [58]:
from torch.utils.data import random_split
DATA_DIR = "archive/data"

# 画像の前処理（リサイズ、テンソル変換など）
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

all_dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform)

TOTAL_SIZE = len(all_dataset)
print(f"TOTAL_SIZE = {TOTAL_SIZE}")
USE_SIZE = TOTAL_SIZE // 2  # 半分のサイズ
print(f"USE_SIZE = {USE_SIZE}")

# データセットをランダムに分割（半分を使用）
dataset, _ = random_split(all_dataset, [USE_SIZE, TOTAL_SIZE - USE_SIZE])

TRAIN_RATO = 0.7
VAL_RATIO = 0.2

train_size = int(USE_SIZE * TRAIN_RATO)
val_size = int(USE_SIZE * VAL_RATIO)
test_size = int(USE_SIZE - train_size - val_size)
split_size = [train_size, val_size, test_size]
print(f"split_size ={split_size}")
print(f"sum = {sum(split_size)}")
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Class = {all_dataset.classes}")
print(f"Class Size = {len(all_dataset.classes)}")

CLASS_SIZE = len(all_dataset.classes)

TOTAL_SIZE = 35626
USE_SIZE = 17813
split_size =[12469, 3562, 1782]
sum = 17813
Class = ['Abra', 'Aerodactyl', 'Alakazam', 'Arbok', 'Arcanine', 'Articuno', 'Beedrill', 'Bellsprout', 'Blastoise', 'Bulbasaur', 'Butterfree', 'Caterpie', 'Chansey', 'Charizard', 'Charmander', 'Charmeleon', 'Clefable', 'Clefairy', 'Cloyster', 'Cubone', 'Dewgong', 'Diglett', 'Ditto', 'Dodrio', 'Doduo', 'Dragonair', 'Dragonite', 'Dratini', 'Drowzee', 'Dugtrio', 'Eevee', 'Ekans', 'Electabuzz', 'Electrode', 'Exeggcute', 'Exeggutor', 'Farfetchd', 'Fearow', 'Flareon', 'Gastly', 'Gengar', 'Geodude', 'Gloom', 'Golbat', 'Goldeen', 'Golduck', 'Golem', 'Graveler', 'Grimer', 'Growlithe', 'Gyarados', 'Haunter', 'Hitmonchan', 'Hitmonlee', 'Horsea', 'Hypno', 'Ivysaur', 'Jigglypuff', 'Jolteon', 'Jynx', 'Kabuto', 'Kabutops', 'Kadabra', 'Kakuna', 'Kangaskhan', 'Kingler', 'Koffing', 'Krabby', 'Lapras', 'Lickitung', 'Machamp', 'Machoke', 'Machop', 'Magikarp', 'Magmar', 'Magnemite', 'Magneton', 'Mankey', 'Marowak', 'Meowth', 'Me

In [53]:
#データ分布の確認
import matplotlib.pyplot as plt
from tqdm import tqdm

def show_distribution(loader, dataset):
    count = [0] * len(dataset.classes)
    for _, label in tqdm(loader, desc="Processing"):
        for l in label:
            count[l] += 1
    plt.bar(dataset.classes, count)
    plt.xticks(rotation=90)
    plt.show()

show_distribution(train_loader, all_dataset)
show_distribution(test_loader, all_dataset)

Processing:  20%|█▉        | 77/390 [00:18<01:21,  3.83it/s]c:\Users\syu\Learning\Class_DL-Exercises\.venv\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing:  53%|█████▎    | 205/390 [00:52<00:46,  3.94it/s]


OSError: image file is truncated

In [59]:
#モデルのロード
import torch.nn as nn
import torchvision.models as models

model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, CLASS_SIZE) #最終層の出力を変更

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [60]:
EPOCH = 1

model.train()

for epoch in tqdm(range(EPOCH), desc="Epoch"):
    # 訓練フェーズ
    model.train()
    total_loss = 0
    for inputs, labels in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch: {epoch}, Training Loss: {avg_train_loss}")

    # 検証フェーズ
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    avg_val_loss = total_loss / len(val_loader)
    print(f"Epoch: {epoch}, Validation Loss: {avg_val_loss}")

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#